In [23]:
from Submission.utils import *
import plotly.express as px

In [24]:
class LandmarkAeroplane():

    def __init__(self, init_pos, At, Bt, Ct, Qt, Rt, Rt_landmark, landmark_pos):

        self.state = init_pos
        self.At = At
        self.Bt = Bt
        self.Ct = Ct
        self.Qt = Qt
        self.Rt = Rt
        self.R_landmark = Rt_landmark
        self.landmarks = landmark_pos
        self.belief = None

    def fetchState(self):
        return self.state
    
    def updateState(self, u):
        self.state = np.matmul(self.At, self.state) + np.matmul(self.Bt, u) + np.random.multivariate_normal(np.zeros(self.Qt.shape[0]),self.Qt)

    def sensor_outputs(self):
        return np.matmul(self.Ct, self.state) + np.random.multivariate_normal(np.zeros(self.Rt.shape[0]),self.Rt)

    def getObservation(self):
        observation, landmark = self.evalH(self.state)
        observation = observation + np.random.multivariate_normal(np.zeros(self.R_landmark.shape[0]),self.R_landmark)
        assert abs(observation[0]) != np.inf or abs(observation[1]) != np.inf or abs(observation[2]) != np.inf
        return observation, landmark
    
    def evalH(self, x):
        dist = [np.linalg.norm(landmark - self.state[:3]) for landmark, limit in (self.landmarks)]
        best_landmark = np.argmin(dist)
        if dist[best_landmark] > self.landmarks[best_landmark][1]:
            return x[:4], -1
        else:
            return np.concatenate([x[:3], np.array([np.linalg.norm(self.landmarks[best_landmark][0] - x[:3])])]), best_landmark
    
    def getJacobian(self, x):
        H = np.zeros((4,6))
        H[:3,:3] = np.eye(3)
        dist = [np.linalg.norm(landmark - self.state[:3]) for landmark, limit in (self.landmarks)]
        best_landmark = np.argmin(dist)
        H[3,:3] = (x[:3] - self.landmarks[best_landmark][0])/np.linalg.norm(x[:3] - self.landmarks[best_landmark][0])
        return H

In [25]:
class ExtendedKalmannFilter():

    def __init__(self, agent, init_belief):
        agent.belief = init_belief
        self.agent = agent
        self.belief = init_belief
    
    def predictionUpdate(self, u):
        # Prediction update of the state of the aeroplane
        # u is the control input
        self.belief['mu'] = np.matmul(self.agent.At, self.belief['mu']) + np.matmul(self.agent.Bt, u) 
        self.belief['sigma'] = np.matmul(self.agent.At, np.matmul(self.belief['sigma'], self.agent.At.T)) + self.agent.Qt
        
    def measurementUpdate(self, z):
    
        # Measurement update of the state of the aeroplane
        # z is the measurement
    
        Kt = np.matmul(np.matmul(self.belief['sigma'], self.agent.Ct.T), np.linalg.inv(np.matmul(self.agent.Ct, np.matmul(self.belief['sigma'], self.agent.Ct.T)) + self.agent.Rt))
        self.belief['mu'] = self.belief['mu'] + np.matmul(Kt, z - np.matmul(self.agent.Ct, self.belief['mu']))
        self.belief['sigma'] = np.matmul(np.identity(self.belief['sigma'].shape[0]) - np.matmul(Kt, self.agent.Ct), self.belief['sigma'])

    def extendedMeasurementUpdate(self):
        z, landmark = self.agent.getObservation()
        if landmark == -1: #No landmark within limits
            z = self.agent.sensor_outputs()
            self.measurementUpdate(z[:3])
        else: #Landmark within limits
            H = self.agent.getJacobian(self.belief['mu'])
            inter1 = np.linalg.inv(np.matmul(H, np.matmul(self.belief['sigma'], H.T)) + self.agent.R_landmark)
            Kt = np.matmul(np.matmul(self.belief['sigma'], H.T), inter1)
            self.belief['mu'] = self.belief['mu'] + np.matmul(Kt, z - self.agent.evalH(self.belief['mu'])[0])
            self.belief['sigma'] = np.matmul(np.identity(self.belief['sigma'].shape[0]) - np.matmul(Kt, H), self.belief['sigma'])

    def extendedPredictionUpdate(self, u):
        self.predictionUpdate(u)
    
    def extendedUpdateBelief(self, u = None, z = None):
        if u is not None:
            self.extendedPredictionUpdate(u)
        if z is not None:
            self.extendedMeasurementUpdate()
        else:
            print("No update provided")

def getIncrementLandmark(t):
    return np.array([-0.128*np.cos(0.032*t), -0.128*np.sin(0.032*t), 0.01])

In [26]:
A = np.array([[1,0,0,1,0,0],[0,1,0,0,1,0],[0,0,1,0,0,1],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]])
B = np.array([[0,0,0],[0,0,0],[0,0,0],[1,0,0],[0,1,0],[0,0,1]])
C = np.array([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0]])
rx, ry, rz = 0.01, 0.01, 0.01
rvx, rvy, rvz  = 0.01, 0.01, 0.01
Qt = np.diag([rx, ry, rz, rvx, rvy, rvz])**2
Rt = np.eye(3)*(10**2)
Rt_landmark = np.eye(4)*(10**2)
Rt_landmark[3,3] = 0.0001
mu0 = np.array([100,0,0,0,4,0])
sigma0 = np.eye(6)*((0.01)**2)
simulation_iterations = 200

In [27]:
def simulateLandmark(Agent,mu0, sigma0, simulation_iterations = 500, leave_obs_cond = lambda i : i < -1):
    AgentEstimator = ExtendedKalmannFilter(Agent, {'mu':mu0, 'sigma':sigma0})
    true_state_list = [Agent.state]
    observed_list = [Agent.getObservation()]
    estimated_list = [mu0]
    belief_covariances = [sigma0]
    beliefs = [mu0]
    for i in range(1,simulation_iterations+1):
        try:
            Agent.updateState(u = getIncrementLandmark(i))

            if leave_obs_cond(i):
                AgentEstimator.extendedUpdateBelief(u = getIncrementLandmark(i))
            else:
                AgentEstimator.extendedUpdateBelief(u = getIncrementLandmark(i), z = 1)
        except:
            print(beliefs)
            raise
        Agent.belief = AgentEstimator.belief
        true_state_list.append(Agent.state)
        observed_list.append(Agent.getObservation())
        estimated_list.append(Agent.belief['mu'])
        belief_covariances.append(Agent.belief['sigma'])
    return true_state_list, observed_list, estimated_list, belief_covariances

In [28]:
landmarks = [(np.array((150, 0, 100)),90), (np.array((-150, 0, 100)),90), (np.array((0, 150, 100)),90) , (np.array((0, -150, 100)),90)]
Agent = LandmarkAeroplane(mu0, A, B, C, Qt, Rt, Rt_landmark, landmarks)
true_state_list, observed_list, estimated_list, belief_covariances = simulateLandmark(Agent, mu0, sigma0, simulation_iterations)

In [31]:
[(np.array((150, 0, 100)),90), (np.array((-150, 0, 100)),90), (np.array((0, 150, 100)),90) , (np.array((0, -150, 100)),90)]

array([1, 2, 3])

In [37]:
tp = [(true_state_list, 'True State'), ([observed_list[i][0] for i in range(len(observed_list))], 'Observed State'), (estimated_list, 'Estimated State')]
plt = plot_trajectories(tp, 0,1,2)
plt.write_html('Q3.html')

In [39]:
plt.add_trace(go.Scatter3d(
        mode='markers',
        x=[150,-150,0,0],
        y=[0,0,150,-150],
        z=[100,100,100,100],
        marker=dict(color=px.colors.qualitative.D3,size=[90, 90, 90,90],sizemode='diameter'
        )
    )
)

In [117]:
def distance_metric(estimated, true):
    return np.linalg.norm(estimated - true)/np.sqrt(len(estimated))

In [118]:
distance_metric(np.array(estimated_list), np.array(true_state_list))

3.1675709625356165

In [119]:
def hungarian_method(cost_matrix):
    cost_matrix = np.array(cost_matrix)
    num_rows, num_cols = cost_matrix.shape

    for i in range(num_rows):
        min_val = np.min(cost_matrix[i, :])
        cost_matrix[i, :] -= min_val

    for j in range(num_cols):
        min_val = np.min(cost_matrix[:, j])
        cost_matrix[:, j] -= min_val

    print(cost_matrix)

    row_covered = np.zeros(num_rows, dtype=bool)
    col_covered = np.zeros(num_cols, dtype=bool)
    cover_zeros(cost_matrix, row_covered, col_covered)
    print(np.sum(row_covered))

    while np.sum(row_covered) < num_rows:
        min_uncovered_val = find_min_uncovered_value(cost_matrix, row_covered, col_covered)
        cost_matrix -= min_uncovered_val
        cover_zeros(cost_matrix, row_covered, col_covered)

    return np.argwhere(cost_matrix == 0)

def cover_zeros(cost_matrix, row_covered, col_covered):
    num_rows, num_cols = cost_matrix.shape

    for i in range(num_rows):
        for j in range(num_cols):
            if cost_matrix[i, j] == 0 and not row_covered[i] and not col_covered[j]:
                row_covered[i] = True
                col_covered[j] = True
                break

def find_min_uncovered_value(cost_matrix, row_covered, col_covered):
    num_rows, num_cols = cost_matrix.shape
    min_val = float('inf')

    for i in range(num_rows):
        for j in range(num_cols):
            if not row_covered[i] and not col_covered[j]:
                min_val = min(min_val, cost_matrix[i, j])

    return min_val

In [121]:
arr = hungarian_method([[1,2,3],[4,5,6],[7,8,9]])

[[0 0 0]
 [0 0 0]
 [0 0 0]]
3


In [123]:
from scipy.optimize import linear_sum_assignment

In [124]:
linear_sum_assignment([[1,2,3],[4,5,6],[7,8,9]])

(array([0, 1, 2]), array([0, 1, 2]))

In [ ]:
import plotly.graph_objects as go

fig = px.line_3d(
    data=[go.Line(x=np.array(true_state_list)[:,0], y=np.array(true_state_list)[:,1], z = np.array(true_state_list)[:,2])],
    layout=go.Layout(
        title=go.layout.Title(text="A Figure Specified By A Graph Object")
    )
)

fig.show()

In [16]:
fig = dict({
    "data": [{"type": "line_3d",
              "x": np.array(true_state_list)[:,0],
              "y": np.array(true_state_list)[:,1]}],
    "layout": {"title": {"text": "A Figure Specified By Python Dictionary"}}
})

# To display the figure defined by this dict, use the low-level plotly.io.show function
import plotly.io as pio

pio.show(fig)

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [{'type': 'line', 'x': array([ 100.        ,  100.00392091,   99.8721543 ,   99.61915049,
         99.23926502,   98.75671241,   98.11843962,   97.35475938,
         96.47619226,   95.45215113,   94.34665772,   93.1364379 ,
         91.80943128,   90.37629958,   88.79556422,   87.11101572,
         85.32580454,   83.42976735,   81.44053005,   79.34862426,
         77.15491534,   74.8581214 ,   72.45740293,   69.97451563,
         67.41019992,   64.76550965,   62.01661315,   59.18557908,
         56.27948499,   53.28016282,   50.21162478,   47.06691212,
         43.84907269,   40.57668441,   37.22551259,   33.80264645,
         30.34941901,   26.83014774,   23.2705912 ,   19.6481627 ,
         15.99977496,   12.32581976,    8.62592528,    4.88444225,
          1.12908014,   -2.63745437,   -6.38556826,  -10.16603697,
        -13.9323358 ,  -17.70725234,  -21.49877479,  -25.25266475,

array([ 100.        ,  100.00392091,   99.8721543 ,   99.61915049,
         99.23926502,   98.75671241,   98.11843962,   97.35475938,
         96.47619226,   95.45215113,   94.34665772,   93.1364379 ,
         91.80943128,   90.37629958,   88.79556422,   87.11101572,
         85.32580454,   83.42976735,   81.44053005,   79.34862426,
         77.15491534,   74.8581214 ,   72.45740293,   69.97451563,
         67.41019992,   64.76550965,   62.01661315,   59.18557908,
         56.27948499,   53.28016282,   50.21162478,   47.06691212,
         43.84907269,   40.57668441,   37.22551259,   33.80264645,
         30.34941901,   26.83014774,   23.2705912 ,   19.6481627 ,
         15.99977496,   12.32581976,    8.62592528,    4.88444225,
          1.12908014,   -2.63745437,   -6.38556826,  -10.16603697,
        -13.9323358 ,  -17.70725234,  -21.49877479,  -25.25266475,
        -29.02450065,  -32.77910567,  -36.51464736,  -40.23023116,
        -43.92462793,  -47.58106169,  -51.23303203,  -54.82328